In [175]:
import pandas as pd 
import os

#loading .bam files
files = [i for i in os.listdir("/home/cusureau/Documents/explore/") if i[-4:] == ".bam"]
col_names = ["chr", "start", "stop", "name","strand","read_count", "covered_bases", "total_lenght","ratio"]

df_tot = pd.read_table(files[0], error_bad_lines=False, names = col_names[:-4] + ["1_" + i for i in col_names[-4:]], usecols = [0,1,2,3,5,12,13,14,15] )

In [176]:
### merge on "name" feature
for indice, file in enumerate(files[1:]):
    df2 = pd.read_table(file, error_bad_lines=False, names = [col_names[3]] + ["{}_".format(indice+2) + i for i in col_names[-4:]], usecols = [3,12,13,14,15])                                                                             
    df_tot = df_tot.merge(df2, left_on = "name", right_on = "name")

In [177]:
#managing column names and order

for i in df_tot.columns[:-3] :
    if i[-6:] == "lenght":
        df_tot = df_tot.drop(labels=i, axis = 1)
                             
df_tot = df_tot.rename(columns={"8_total_lenght":"total_lenght_kb"})

#Je créer la colonne correspondant à la longueur des features en .kb et je réorganise les colonnes
df_tot["total_lenght_kb"] = df_tot["total_lenght_kb"].div(1000)
df_tot = df_tot[['chr', 'start', 'stop', 'name',
       'total_lenght_kb', 'strand', '1_read_count',
       '1_covered_bases', '1_ratio', '2_read_count', '2_covered_bases',
       '2_ratio', '3_read_count', '3_covered_bases', '3_ratio', '4_read_count',
       '4_covered_bases', '4_ratio', '5_read_count', '5_covered_bases',
       '5_ratio', '6_read_count', '6_covered_bases', '6_ratio', '7_read_count',
       '7_covered_bases', '7_ratio', '8_read_count', '8_covered_bases', '8_ratio']]

df_tot = df_tot.set_index("name")

In [178]:
#counting millions of reads per samples
total_reads_per_samples = {}

for i, j  in enumerate(df_tot.columns):
    if j[2:] == "read_count" : 
        total_reads_per_samples[j[0]] = sum(df_tot[j]) / 1000000

In [179]:
#Je créer un nouveau dictionnaire avec le read_count normalisé par le nombre de read totaux. 
normalized_read_count_per_millions = {'name':df_tot.index}

for i in df_tot.columns :
    
    if i[2:] == "read_count":
        normalized_values = []
        sample_name=i[0]
        for value in df_tot[i]:
            normalized_values.append(value/total_reads_per_samples[sample_name])
        
        normalized_read_count_per_millions[sample_name] = normalized_values

#Je créer la dataframe rpm (read per million)

rpm = pd.DataFrame(normalized_read_count_per_millions)
rpm = rpm.set_index("name")

In [185]:
#je normalise la df rpm avec la longueur de chaque lnc (calcul du rpkm)

rpkm_dic = {'name':df_tot.index}
lnc_lenght = df_tot["total_lenght_kb"]
for i in rpm.columns:
    temp_rpm = rpm[i]
    temp_rpkm = temp_rpm/lnc_lenght
    rpkm_dic[i] = temp_rpkm

#je créer la dataframe rpkm (read per kilobase Million)
rpkm = pd.DataFrame(rpkm_dic)
rpkm = rpkm.set_index("name")

name
LINC01725:44       59.031
lnc-ZFHX3-27:11    23.130
lnc-GPR119-1:1     47.189
CACNA1G-AS1:3       2.237
PIK3CD-AS2:1        7.775
                    ...  
LINC02589:3        24.187
TRIM52-AS1:36       3.356
LINC01995:3         8.835
lnc-LSMEM1-3:4     85.692
MIR3681HG:37       76.503
Name: total_lenght_kb, Length: 107039, dtype: float64


In [189]:
rpkm

,1,2,3,4,5,6,7,8
name,,,,,,,,
LINC01725:44,0.002518,0.024949,0.008322,0.006782,0.006130,0.006551,0.064970,0.002989
lnc-ZFHX3-27:11,0.019280,0.030068,0.013004,0.010715,0.023467,0.038666,0.055271,0.003814
lnc-GPR119-1:1,0.092137,0.118555,0.159345,0.098776,0.104960,0.097835,0.178959,0.120573
CACNA1G-AS1:3,0.000000,0.036576,0.000000,0.008522,0.040440,0.064831,0.032970,0.000000
PIK3CD-AS2:1,0.004780,0.028940,0.014184,0.019616,0.017453,0.018653,0.028458,0.034037
...,...,...,...,...,...,...,...,...
LINC02589:3,0.307283,0.275702,0.276065,0.296753,0.302959,0.263828,0.194650,0.300887
TRIM52-AS1:36,0.653313,1.124548,1.332390,1.048062,0.606516,0.972325,1.882689,1.143402
LINC01995:3,0.033649,0.059039,0.038583,0.026972,0.035838,0.027359,0.077915,0.024961


In [201]:
#Median of ratios method 
df_read_count = df_tot[["1_read_count","2_read_count","3_read_count","4_read_count","5_read_count","6_read_count","7_read_count","8_read_count"]]

In [273]:
df_read_count.to_csv("read_count.csv")

In [202]:
#Calcul de la moyenne géométrique de chaque feature (pseudo_reference_sample)
from scipy.stats.mstats import gmean  
geo_mean =  []
for i in range(len(df_read_count.index)):
    geo_mean.append(gmean(df_read_count.iloc[i]))

In [216]:
#Je divise le nombre de read par la 'pseudo reference' pour avoir un ratio. 
ratio = {'name':df_tot.index}

for i in df_read_count.columns[:-1]:   
    temp_ratio = df_read_count[i]/geo_mean
    ratio[i] = temp_ratio

In [217]:
df_ratio = pd.DataFrame(ratio)
df_ratio = df_ratio.set_index("name")

In [242]:
#Je regarde la median du ratio par échantillon. 
import scipy 
normalization_factors = {}

for i in df_ratio.columns:
    normalization_factors[i] = scipy.nanmedian(df_ratio[i])

/home/cusureau/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: scipy.nanmedian is deprecated and will be removed in SciPy 2.0.0, use numpy.nanmedian instead
  """


In [266]:
#Normalized counts : je divise le nombre de read par le ratio median par échantillon. 
normalized_counts = {'name':df_tot.index}

for i in df_read_count.columns:
    normalized_counts[i] = df_read_count[i]/normalization_factors[i]
    
normalized_df = pd.DataFrame(normalized_counts)
normalized_df = normalized_df.set_index('name')

In [267]:
normalized_df

,1_read_count,2_read_count,3_read_count,4_read_count,5_read_count,6_read_count,7_read_count,8_read_count
name,,,,,,,,
LINC01725:44,7.934484,53.563797,24.090041,19.969491,18.084358,20.266602,86.136816,9.708917
lnc-ZFHX3-27:11,23.803453,25.294015,14.749005,12.362066,27.126537,46.866518,28.712272,4.854459
lnc-GPR119-1:1,232.083667,203.468036,368.725121,232.501930,247.529649,241.932566,189.666644,313.112577
CACNA1G-AS1:3,0.000000,2.975767,0.000000,0.950928,4.521089,7.599976,1.656477,0.000000
PIK3CD-AS2:1,1.983621,8.183358,5.407968,7.607425,6.781634,7.599976,4.969432,14.563376
...,...,...,...,...,...,...,...,...
LINC02589:3,396.724217,242.524972,327.427907,358.024445,366.208248,334.398939,105.738464,400.492831
TRIM52-AS1:36,117.033644,137.257231,219.268539,175.446242,101.724513,170.999458,141.904883,211.168947
LINC01995:3,15.868969,18.970512,16.715539,11.886602,15.823813,12.666626,15.460454,12.136146


In [274]:
df_tot

,chr,start,stop,total_lenght_kb,strand,1_read_count,1_covered_bases,1_ratio,2_read_count,2_covered_bases,...,5_ratio,6_read_count,6_covered_bases,6_ratio,7_read_count,7_covered_bases,7_ratio,8_read_count,8_covered_bases,8_ratio
name,,,,,,,,,,,,,,,,,,,,,
LINC01725:44,1,83801515,83860546,59.031,-,4,349,0.005912,144,7164,...,0.017465,16,1010,0.017110,312,17754,0.300757,4,387,0.006556
lnc-ZFHX3-27:11,16,74192391,74215521,23.130,-,12,826,0.035711,68,3400,...,0.057674,37,1513,0.065413,104,6855,0.296368,2,129,0.005577
lnc-GPR119-1:1,X,130477068,130524257,47.189,-,117,2609,0.055288,547,7805,...,0.105597,191,3651,0.077370,687,11311,0.239696,129,2659,0.056348
CACNA1G-AS1:3,17,50557173,50559410,2.237,-,0,0,0.000000,8,376,...,0.129191,6,322,0.143943,6,484,0.216361,0,0,0.000000
PIK3CD-AS2:1,1,9679799,9687574,7.775,-,1,66,0.008489,22,1249,...,0.049775,6,409,0.052604,18,856,0.110096,6,292,0.037556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LINC02589:3,X,103530766,103554953,24.187,-,200,8048,0.332741,652,19080,...,0.518750,264,10445,0.431844,383,14265,0.589780,165,6902,0.285360
TRIM52-AS1:36,5,181261211,181264567,3.356,+,59,1167,0.347735,369,2356,...,0.586114,135,1741,0.518772,514,2362,0.703814,87,1015,0.302443
LINC01995:3,3,182498289,182507124,8.835,-,8,646,0.073118,51,2344,...,0.126542,10,682,0.077193,56,3615,0.409168,5,118,0.013356
